<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/3step_lecture/Training_NN_with_numerical_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
tf.__version__

'2.1.0'

### Utility Function

In [0]:
epsilon = 0.0001

def _t(x):
  return np.transpose(x)

def _m(A, B):
  return np.matmul(A, B)

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def MSE(h, y):
  return 1 / 2 * np.mean(np.square(h - y))

### Make Neuron

In [0]:
class Neuron:
  def __init__(self, W, b, a):
    # Model Parameter
    self.W = W
    self.b = b
    self.a = a

    # Gradients
    self.dW = np.zeros_like(self.W)
    self.db = np.zeros_like(self.b)

  def __call__(self, x):
    return self.a(_m(_t(self.W), x) + self.b)

### Make DNN

In [0]:
class DNN:
  def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
    def init_var(i, o):
      return np.random.normal(0., 0.01, (i, o)), np.zeros((o,))
    
    self.sequence = list()
    # First hidden layer
    W, b = init_var(num_input, num_neuron)
    self.sequence.append(Neuron(W, b, activation))

    # Hidden layers
    for _ in range(hidden_depth - 1):
      W, b = init_var(num_neuron, num_neuron)
      self.sequence.append(Neuron(W, b, activation))

    # Output layer
    W, b = init_var(num_neuron, num_output)
    self.sequence.append(Neuron(W, b, activation))


  def __call__(self, x):
    for layer in self.sequence:
      x = layer(x)
    return x
    
  def calc_gradient(self, x, y, loss_func):
    def get_new_sequence(layer_index, new_neuron):
      new_sequence = list()
      for i, layer in enumerate(self.sequence):
        if i == layer_index:
          new_sequence.append(new_neuron)
        else:
          new_sequence.append(layer)
      return new_sequence
    
    def eval_sequence(x, sequence):
      for layer in self.sequence:
        x = layer(x)
      return x

    loss = loss_func(self(x), y)

    for layer_id, layer in enumerate(self.sequence): # iterate layer
      for w_i, w in enumerate(layer.W): # iterate W (row)
        for w_j, ww in enumerate(w): # iterate W (col)
          W = np.copy(layer.W)
          W[w_i][w_j] = ww + epsilon

          new_neuron = Neuron(W, layer.b, layer.a)
          new_seq = get_new_sequence(layer_id, new_neuron)
          h = eval_sequence(x, new_seq)

          num_grad = (loss_func(h, y) - loss) / epsilon
          layer.dW[w_i][w_j] = num_grad
      
      for b_i, bb in enumerate(layer.b):
        b = np.copy(layer.b)
        b[b_i] = bb + epsilon

        new_neuron = Neuron(layer.W, b, layer.a)
        new_seq = get_new_sequence(layer_id, new_neuron)
        h = eval_sequence(x, new_seq)

        num_grad = (loss_func(h, y) - loss) / epsilon
        layer.db[b_i] = num_grad
    
    return loss

### Define GD

In [0]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
  loss = network.calc_gradient(x, y, loss_obj)
  for layer in network.sequence:
    layer.W += -alpha * layer.dW
    layer.b += -alpha * layer.db
  return loss

### Test

In [15]:
x = np.random.normal(0., 1., (10, ))
y = np.random.normal(0., 1., (2, ))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
  loss = gradient_descent(dnn, x, y, MSE, 0.01)
  print('Epoch {}, Test Loss: {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0, Test Loss: 1.2468301254638356
Epoch 1, Test Loss: 1.2468301254638356
Epoch 2, Test Loss: 1.2468301254638356
Epoch 3, Test Loss: 1.2468301254638356
Epoch 4, Test Loss: 1.2468301254638356
Epoch 5, Test Loss: 1.2468301254638356
Epoch 6, Test Loss: 1.2468301254638356
Epoch 7, Test Loss: 1.2468301254638356
Epoch 8, Test Loss: 1.2468301254638356
Epoch 9, Test Loss: 1.2468301254638356
Epoch 10, Test Loss: 1.2468301254638356
Epoch 11, Test Loss: 1.2468301254638356
Epoch 12, Test Loss: 1.2468301254638356
Epoch 13, Test Loss: 1.2468301254638356
Epoch 14, Test Loss: 1.2468301254638356
Epoch 15, Test Loss: 1.2468301254638356
Epoch 16, Test Loss: 1.2468301254638356
Epoch 17, Test Loss: 1.2468301254638356
Epoch 18, Test Loss: 1.2468301254638356
Epoch 19, Test Loss: 1.2468301254638356
Epoch 20, Test Loss: 1.2468301254638356
Epoch 21, Test Loss: 1.2468301254638356
Epoch 22, Test Loss: 1.2468301254638356
Epoch 23, Test Loss: 1.2468301254638356
Epoch 24, Test Loss: 1.2468301254638356
Epoch 25, 